# 项目描述

# 在word2vec上训练情感分析模型

In [14]:
import os
import re
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

from nltk.corpus import stopwords

from gensim.models.word2vec import Word2Vec

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans

### 和之前的操作一致

In [7]:
def load_dataset(name, nrows=None):
    datasets = {
        'unlabeled_train': 'unlabeledTrainData.tsv',
        'labeled_train': 'labeledTrainData.tsv',
        'test': 'testData.tsv'
    }
    if name not in datasets:
        raise ValueError(name)
    data_file = os.path.join('..', 'data', datasets[name])
    df = pd.read_csv(data_file, sep='\t', escapechar='\\', nrows=nrows)
    print('Number of reviews: {}'.format(len(df)))
    return df

In [8]:
eng_stopwords = set(stopwords.words('english'))

def clean_text(text, remove_stopwords=False):
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = text.lower().split()
    if remove_stopwords:
        words = [w for w in words if w not in eng_stopwords]
    return words

### 读入之前训练好的Word2Vec模型

In [9]:
# 重新加载模型
model_name = '300features_40minwords_10context.model'
model = Word2Vec.load(os.path.join('..', 'models', model_name))

### 我们可以根据word2vec的结果去对影评文本进行编码

编码方式有一点粗暴，简单说来就是把这句话中的词的词向量做平均

In [10]:
df = load_dataset('labeled_train')
df.head()

Number of reviews: 25000


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"""The Classic War of the Worlds"" by Timothy Hin..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [11]:
def to_review_vector(review):
    words = clean_text(review, remove_stopwords=True)
    array = np.array([model[w] for w in words if w in model])
    return pd.Series(array.mean(axis=0))

In [12]:
train_data_features = df.review.apply(to_review_vector)
train_data_features.head()

d:\program\python36\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
d:\program\python36\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.000109,-0.009772,-0.013653,-0.007361,-0.007370,0.015015,0.008392,-0.015024,0.001520,-0.016185,...,0.012631,0.003943,-0.022444,0.006767,0.029611,0.016667,0.016060,0.015459,-0.000738,0.003131
1,-0.003318,0.001500,-0.001083,-0.006788,0.005144,0.017002,0.010130,-0.023831,-0.003708,-0.012752,...,-0.011927,0.001700,-0.000500,0.004355,0.031129,0.025301,0.001144,0.002118,0.021805,0.002903
2,0.003225,-0.024197,-0.024707,-0.023636,0.002737,0.014117,0.031443,0.003175,-0.014963,-0.001365,...,0.010130,0.010490,-0.011819,-0.000962,0.005017,0.019952,-0.002238,-0.009409,0.009671,0.015794
3,-0.003529,0.010859,-0.024873,-0.023327,-0.007675,0.014748,0.007054,-0.011101,0.006327,-0.006325,...,-0.007549,0.009034,0.003849,-0.003576,0.013312,0.011763,-0.005736,-0.018785,0.007683,0.015764
4,0.007099,-0.022823,-0.021836,-0.017857,-0.011008,0.011776,0.021602,0.002612,-0.003603,-0.015882,...,-0.002162,0.016925,0.003863,-0.007090,0.000416,0.010169,0.008147,-0.013053,0.008776,0.012196


### 用随机森林构建分类器

In [13]:
forest = RandomForestClassifier(n_estimators = 100, random_state=42)
forest = forest.fit(train_data_features, df.sentiment)

##### 同样在训练集上试试，确保模型能正常work

In [15]:
accuracy_score(df.sentiment, forest.predict(train_data_features))

1.0

### 清理占用内容的变量

In [8]:
del df
del train_data_features

### 预测测试集结果并上传kaggle

In [16]:
df = load_dataset('test')
df.head()

Number of reviews: 25000


,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [17]:
test_data_features = df.review.apply(to_review_vector)
test_data_features.head()

d:\program\python36\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
d:\program\python36\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.003155,-0.008708,-0.012843,-0.023635,-0.013472,0.035907,0.031323,-0.002444,0.006864,-0.036429,...,-0.005040,0.021943,0.004438,0.000726,0.006226,-0.000798,-0.004117,-0.012614,0.012314,0.004205
1,0.000962,-0.008366,-0.018177,-0.016800,-0.016402,0.021164,0.008236,-0.012354,-0.005438,-0.017450,...,0.018589,0.016258,0.001767,-0.000563,0.024899,0.023688,0.001109,0.000012,0.013827,-0.004249
2,0.004503,0.009341,-0.005498,-0.018236,-0.020221,0.026734,0.001121,-0.004303,0.010391,-0.028769,...,-0.003054,0.008920,-0.001149,-0.002643,0.027320,0.004598,-0.003209,0.007516,0.013600,0.006372
3,-0.014232,0.010079,-0.015144,-0.031654,-0.016407,0.030890,0.019545,-0.012133,0.003996,-0.007367,...,0.003917,0.026128,0.000522,-0.011671,0.024614,0.021016,0.007773,0.001402,-0.003772,0.006269
4,-0.003966,-0.010148,-0.012012,-0.018483,-0.003455,0.024889,0.015580,-0.007595,-0.009080,-0.030942,...,0.005784,0.008077,-0.019053,0.014554,0.005411,0.033621,-0.007050,-0.004373,0.012797,-0.001821


In [18]:
result = forest.predict(test_data_features)
output = pd.DataFrame({'id':df.id, 'sentiment':result})
output.to_csv(os.path.join('..', 'data', 'Word2Vec_model.csv'), index=False)
output.head()

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,1
3,7186_2,0
4,12128_7,1


In [ ]:
del df
del test_data_features
del forest

------------------
### 对词向量进行聚类研究和编码
使用Kmeans进行聚类

In [19]:
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] // 10

AttributeError: 'Word2Vec' object has no attribute 'syn0'

In [6]:
%%time

kmeans_clustering = KMeans(n_clusters = num_clusters, n_jobs=4)
idx = kmeans_clustering.fit_predict(word_vectors)

CPU times: user 2.03 s, sys: 377 ms, total: 2.41 s
Wall time: 13min 19s


In [7]:
word_centroid_map = dict(zip(model.index2word, idx))

In [8]:
import pickle

filename = 'word_centroid_map_10avg.pickle'
with open(os.path.join('..', 'models', filename), 'bw') as f:
    pickle.dump(word_centroid_map, f)
    
#with open(os.path.join('..', 'models', filename), 'br') as f:
#    word_centroid_map = pickle.load(f)    

### 输出一些clusters看

In [9]:
for cluster in range(0,10):
    print("\nCluster %d" % cluster)
    print([w for w,c in word_centroid_map.items() if c == cluster])


Cluster 0
['praised', 'appreciated', 'noted', 'avoided', 'criticized', 'admired']

Cluster 1
['misfit', 'con', 'hoodlum', 'spy', 'rogue']

Cluster 2
['contrasts', 'healthy', 'glamour', 'eroticism', 'sensual']

Cluster 3
['matthew', 'kingsley', 'klein', 'hackman', 'meyers', 'perry', 'simpson', 'pullman', 'dana', 'olsen', 'ryan', 'barrie', 'caan', 'tho', 'farina', 'stiller', 'hutton', 'sparks', 'lillard', 'broderick', 'kline', 'reprise', 'mcconaughey', 'carvey', 'harrelson']

Cluster 4
['wolves', 'papillon', 'continent']

Cluster 5
['tick', 'drain', 'nailed', 'puke', 'boil', 'stalk']

Cluster 6
['cotton', 'denver', 'windsor', 'marsh', 'bell']

Cluster 7
['lighting', 'costumes', 'sfx', 'props', 'design', 'costuming', 'designs', 'makeup']

Cluster 8
['decline', 'swashbuckling', 'swashbuckler', 'prestige', 'potboiler', 'latter', 'glory', 'untouchables', 'fame']

Cluster 9
['slashed', 'butchered', 'mutilated', 'eaten', 'slaughtered', 'continually']


### 把评论数据转成cluster bag vectors

In [11]:
wordset = set(word_centroid_map.keys())

def make_cluster_bag(review):
    words = clean_text(review, remove_stopwords=True)
    return (pd.Series([word_centroid_map[w] for w in words if w in wordset])
              .value_counts()
              .reindex(range(num_clusters+1), fill_value=0))

In [12]:
df = load_dataset('labeled_train')
df.head()

Number of reviews: 25000


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"""The Classic War of the Worlds"" by Timothy Hin..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [13]:
train_data_features = df.review.apply(make_cluster_bag)
train_data_features.head()

,0,1,2,3,4,5,6,7,8,9,...,1296,1297,1298,1299,1300,1301,1302,1303,1304,1305
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,5,0,0,0,0,0,0,0,0


### 再用随机森林算法建模

In [14]:
forest = RandomForestClassifier(n_estimators = 100, random_state=42)
forest = forest.fit(train_data_features, df.sentiment)

##### 在训练集上试一试效果

In [15]:
confusion_matrix(df.sentiment, forest.predict(train_data_features))

array([[12500,     0],
       [    0, 12500]])

#### 去掉无用的占内存的量

In [16]:
del df
del train_data_features

### 载入测试数据做预测

In [17]:
df = load_dataset('test')
df.head()

Number of reviews: 25000


,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [18]:
test_data_features = df.review.apply(make_cluster_bag)
test_data_features.head()

,0,1,2,3,4,5,6,7,8,9,...,1296,1297,1298,1299,1300,1301,1302,1303,1304,1305
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
result = forest.predict(test_data_features)
output = pd.DataFrame({'id':df.id, 'sentiment':result})
output.to_csv(os.path.join('..', 'data', 'Word2Vec_BagOfClusters.csv'), index=False)
output.head()

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,1
3,7186_2,0
4,12128_7,1


In [ ]:
del df
del test_data_features
del forest